In [ ]:
# Initial Setup.
import pandas as pd
import re
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import networkx as nx
from tqdm import tqdm

In [ ]:
#Read in cleaned dataset.
file_path=path+ "fraud_cleaned.csv"
data=pd.read_csv(file_path,converters={'id':str,'entities':str,'interact_with':str,'author_id':str})

<ipython-input-2-b721fe1c8e81>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(file_path,converters={'id':str,'entities':str,'interact_with':str,'author_id':str})


In [ ]:
data_isolates=data[data['interact_with']=='']

In [ ]:
#Filter out those empty rows.
data_new = data[(data['interact_with'].isin(data['author_id']))]

In [ ]:
data_new.loc[data_new['interact_with']=='']

,Unnamed: 0,entities,possibly_sensitive,public_metrics,author_id,conversation_id,text,lang,edit_history_tweet_ids,created_at,id,in_reply_to_user_id,withheld,reference_type,reference_id,reference_author_id,text_y,interact_with


In [ ]:
G = nx.from_pandas_edgelist(data_new,'author_id','interact_with',create_using=nx.DiGraph())
strongly_connected_components = list(nx.strongly_connected_components(G))

In [ ]:
min_size = 2
ec = [i for i in strongly_connected_components  if len(i) >= min_size]
len(ec)

143

In [ ]:
flattened_list = [item for subset in ec for item in subset]

In [ ]:
len(flattened_list)

470

In [ ]:
len(max(strongly_connected_components, key=len))

126

In [ ]:
def clusters(text):
    if text in flattened_list:
      return 1
    else:
      return 0
from tqdm import tqdm
tqdm.pandas()
data_new=data_new.append(data_isolates)
data_new['cluster'] = data_new['author_id'].progress_apply(clusters)
def clusters_s(text):
    if text in flattened_list_s:
      return "strongest"

data_new['strongest'] = data_new['author_id'].progress_apply(clusters_s)

<ipython-input-19-ee9a38c311bc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_new=data_new.append(data_isolates)
100%|██████████| 219872/219872 [00:00<00:00, 613999.06it/s]


In [ ]:
# Clean the text.
def cleaner(x):
    import re
    x = re.sub(r'http\S+', '', x)
    x = re.sub('\n', '',x)
    x = re.sub("@[A-Za-z0-9_]+","", x)
    x=re.sub(r'^RT ', '', x)
    x = re.sub("#[A-Za-z0-9_]+","", x)
    #x = re.sub(r'[^\w\s]', '', x)
    #import nltk
    #from nltk.corpus import stopwords
    #words = nltk.word_tokenize(x)

    # Remove stopwords
    #stop_words = set(stopwords.words('english'))
    #words = [word for word in words if word.lower() not in stop_words]
    #from gensim.parsing.preprocessing import remove_stopwords
    #x = remove_stopwords(x)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    x = emoji_pattern.sub(r'', x)
    x = x.lower()

    #import emoji
    #cleaned = emoji.get_emoji_regexp().sub("", x)

    return x

In [ ]:
def cleaner_hm(x):
    import re
    x = re.sub(r'http\S+', '', x)
    x = re.sub(',', '',x)
    x = re.sub('\?','',x)
    x = re.sub('!','',x)
    x = re.sub('\n', '',x)
    x = re.sub('\.', '',x)
    x=re.sub(r'^RT ', '', x)
    x = re.sub("@[A-Za-z0-9_]+","", x)
    x = re.sub("#[A-Za-z0-9_]+","", x)
    #x = re.sub("@[A-Za-z0-9]+\s",'', x)
    #x = x.replace("#", "").replace("_", "") #Remove hashtag sign but keep the text
    x = re.sub(r'[^\w\s]', '', x)
    #from gensim.parsing.preprocessing import remove_stopwords
    #x = remove_stopwords(x)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    x = emoji_pattern.sub(r'', x)
    x = x.lower()

    #import emoji
    #cleaned = emoji.get_emoji_regexp().sub("", x)

    return x

In [ ]:

all_logical = "actually,admittedly,also,alternatively,although,anyhow,anyway,because,besides,but,cause,caused,causes,causing,consequence,consequences,consequently,correspondingly,enable,enabled,enables,finally,for,fortunately,further,furthermore,hence,however,if,incidentally,instead,likewise,moreover,nevertheless,next,nonetheless,nor,or,otherwise,rather,secondly,similarly,summarizing,then,therefore,thereupon,though,thus,unless,whereas,while".split(",")
all_logical_caveat = "for since so".split(" ")

addition = "and,also,besides,further,furthermore,too,moreover,then,another,indeed,likewise".split(",")
all_negative = "admittedly,alternatively,although,and conversely,anyhow,but,however,nevertheless,nonetheless,nor,or,otherwise,rather,though,unless,until,whenever,whereas,yet".split(",")


#Givenness pronouns:
givenness_prp = "we our us ours ourselves they them their themselves theirs".split(" ")



In [ ]:
def nwords(text):
    text_tokens=text.split(" ")
    text_tokens = [a for a in text_tokens if a!='']
    return len(text_tokens)
def safe_divide(numerator, denominator):
    if denominator == 0:
        index = 0
    else: index = numerator/denominator
    return index
def logical_connectives_counter(text):
    text_tokens=text.split(" ")
    counter=0
    for item in text_tokens:
        if item in (all_logical or all_logical_caveat):
            counter+=1
    return counter
def adversative_connectives_counter(text):
    text_tokens=text.split(" ")
    counter=0
    for item in text_tokens:
        if item in all_negative:
            counter+=1
    return counter
def group_identity_counter(text):
    text_tokens=text.split(" ")
    counter=0
    for item in text_tokens:
        if item in givenness_prp:
            counter+=1
    return counter
def addition_counter(text):
    text_tokens=text.split(" ")
    counter=0
    for item in text_tokens:
        if item in addition:
            counter+=1
    return counter
def logical_connectives_words_ratio(text):
    return safe_divide(logical_connectives_counter(text),nwords(text))
def addition_words_ratio(text):
    return safe_divide(addition_counter(text),nwords(text))
def adversative_connectives_words_ratio(text):
    return safe_divide(adversative_connectives_counter(text),nwords(text))
def group_identity_words_ratio(text):
    return safe_divide(group_identity_counter(text),nwords(text))


In [ ]:
data_new['text_cleaned_hm']=data_new['text'].progress_apply(cleaner_hm)

100%|██████████| 219872/219872 [00:05<00:00, 40172.22it/s]


In [ ]:

data_new['identity'] = data_new['text_cleaned_hm'].apply(group_identity_words_ratio)
data_new['logical'] = data_new['text_cleaned_hm'].apply(logical_connectives_words_ratio)
data_new['negative'] = data_new['text_cleaned_hm'].apply(adversative_connectives_words_ratio)
data_new['addition'] = data_new['text_cleaned_hm'].apply(addition_words_ratio)